In [6]:
"""


當用戶關注Line@後，Line會發一個FollowEvent，

我們接受到之後，取得用戶個資，對用戶綁定自定義菜單，會回傳四個消息給用戶


"""

'\n\n\n當用戶關注Line@後，Line會發一個FollowEvent，\n\n我們接受到之後，取得用戶個資，對用戶綁定自定義菜單，會回傳四個消息給用戶\n\n\n'

In [7]:
"""

啟用伺服器基本樣板

"""

# part1 - 引用套件

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r',encoding="utf-8"))
server_url=secretFileContentJson.get("server_url")



# Part2 伺服器方法準備
# 設定Server啟用細節
# 準備一個 Flask Web Application
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件

# 就是收發小妹啦，他專門跟line溝通
# LineBotApi( channel_access_token )
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))

# 就是總機啦，專門收信
# WebHandler( channel secret )
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
# 這就是 Line發消息的入口
@app.route("/", methods=['POST'])
def callback():
    
    # handler 取出信件的內容
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # get request body as text
    body = request.get_data(as_text=True)
    
    
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        # 把信件表頭跟內容，交給handler做處理
        # 驗證
        # 後續轉發
        # handle( 信件內容, 信件驗證碼)
        handler.handle(body, signature)
        
        # 若信件無效
    except InvalidSignatureError:
        
        # 忽略不看
        abort(400)

    return 'OK'


In [8]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [

    # 寄給用戶的文字消息
    TextSendMessage(text="關注區塊鏈技術，掌握市場脈動。"),
    
    TextSendMessage(text="人類所以充滿驚奇，在於人體那一精密又不可探究的系統。佈滿神經元的大腦，而後延展遍歷人體。\n\n區塊鏈就好比是人體那驚奇的神經系統，社會是我們的人身，在全身佈滿了神經後，造就了不可思量的奧妙生命。\n\n點選菜單，了解區塊鏈前世今生，以文字輸入 more，得到更多資訊。"),
    
    # 寄給用戶的圖片消息
    # original_content_url : 原始圖片
    # preview_image_url : 預覽圖片
    ImageSendMessage(
        original_content_url='https://%s/images/003.jpeg' %server_url ,
        preview_image_url='https://%s/images/001.jpg' %server_url
    ),
    
    ImageSendMessage(original_content_url='https://%s/images/004.png' %server_url,
    preview_image_url='https://%s/images/005.jpg' %server_url)
]

In [9]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent, UnfollowEvent
)

# 載入requests套件
import requests

# 若用戶封鎖我們，則印出用戶的user_id
@handler.add(UnfollowEvent)
def demo_unfollow(event):
    print(event.source.user_id)


# 告知handler，如果收到FollowEvent，則做下面的方法處理
# handler.add()
# 告知handler 收到 xx Event就做下面的方法
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    # 請line_bot_api 取出用戶個資
    # get_profile(用戶的id)
    # event.source.user_id
    # 消息來源的用戶id
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
    # 將用戶資訊存在檔案內
    # save file python
    with open("../users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkResult=line_bot_api.link_rich_menu_to_user(event.source.user_id, linkRichMenuId)
    
    # 回覆文字消息與圖片消息
    # reply_message(event.reply_token, 剛剛做的消息素材)
    # 回復用戶
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [10]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jul/2020 13:41:25] "POST / HTTP/1.1" 200 -


U611ce9f7db42bb73be1bb68e63166bf9


127.0.0.1 - - [26/Jul/2020 13:41:25] "POST / HTTP/1.1" 200 -


U27eaae6bcda9a8cbd34e9a149f35a635


...

127.0.0.1 - - [26/Jul/2020 13:41:46] "GET /images/004.png HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:41:47] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:41:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:41:48] "GET /images/001.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:41:49] "GET /images/005.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:41:57] "POST / HTTP/1.1" 200 -


Ufb7c8d7a32fd14952081f98ab1e917c1


127.0.0.1 - - [26/Jul/2020 13:42:01] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:43:07] "GET /images/001.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:43:07] "GET /images/005.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:43:09] "POST / HTTP/1.1" 200 -


Ufb7c8d7a32fd14952081f98ab1e917c1


127.0.0.1 - - [26/Jul/2020 13:43:12] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:43:13] "GET /images/001.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:43:13] "GET /images/005.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:43:13] "GET /images/003.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:43:13] "GET /images/004.png HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2020 13:43:17] "GET /images/003.jpeg HTTP/1.1" 200 -
